# 🏗️ Data Lakehouse con Parquet, Delta Lake e Iceberg (conceptos y práctica ligera)

Objetivo: comprender los principios de Lakehouse y practicar un flujo básico con Parquet (local) y notas de cómo migrar a Delta Lake o Apache Iceberg.

- Duración: 120 min
- Dificultad: Media/Alta
- Prerrequisitos: Mid 03 (AWS/S3) y 07 (Particionado)

## 1. Lakehouse en pocas líneas

- Tabla de datos en formato columna (Parquet) sobre object storage.
- Transaccionalidad y versiones con capas de metadatos (Delta/Iceberg/Hudi).
- Catálogo central (Glue/Unity/Metastore) y gobernanza integrada.
- Lectores: engines SQL (Athena/Trino/Spark) + ML + BI.

## 2. Hands-on: tabla Parquet particionada local

In [ ]:
import os, time
from pathlib import Path
import pandas as pd

BASE = Path('datasets/processed/lakehouse_demo')
(BASE).mkdir(parents=True, exist_ok=True)

df = pd.DataFrame({
  'id':[1,2,3,4],
  'fecha':['2025-10-01','2025-10-02','2025-10-02','2025-10-03'],
  'producto_id':[101,102,101,103],
  'cantidad':[1,2,1,3],
  'precio':[100.0,50.0,100.0,20.0]
})
df['total'] = df['cantidad'] * df['precio']
df['anio'] = pd.to_datetime(df['fecha']).dt.year
df['mes'] = pd.to_datetime(df['fecha']).dt.strftime('%Y-%m')

for (anio, mes), part in df.groupby(['anio','mes']):
    part_dir = BASE / f'anio={anio}' / f'mes={mes}'
    part_dir.mkdir(parents=True, exist_ok=True)
    fp = part_dir / f'ventas_{int(time.time())}.parquet'
    part.to_parquet(fp, index=False)
str(BASE)

### 2.1 Lectura particionada y pruning manual

In [ ]:
parts = list((BASE / 'anio=2025' / 'mes=2025-10').glob('*.parquet'))
pd.concat([pd.read_parquet(p) for p in parts]).head()

## 3. Delta Lake/Iceberg: cómo y cuándo

- Delta Lake agrega ACID, time travel y MERGE INTO sobre Parquet.
- Iceberg optimiza la gestión de metadatos, particiones ocultas y evolución de esquema.
- Requiere motor como Spark/Trino/Flint y un catálogo (Glue/REST).
- Coste/beneficio: evalúa volumen, concurrencia, latencia y SLA.

### 3.1 Ejemplo Delta Lake (referencia con PySpark) [opcional]

In [ ]:
delta_demo = r'''
from pyspark.sql import SparkSession
spark = (SparkSession.builder
    .appName('DeltaDemo')
    .config('spark.sql.extensions','io.delta.sql.DeltaSparkSessionExtension')
    .config('spark.sql.catalog.spark_catalog','org.apache.spark.sql.delta.catalog.DeltaCatalog')
    .getOrCreate())

df = spark.read.parquet('s3://bucket/curated/ventas/')
df.write.format('delta').mode('overwrite').save('s3://bucket/delta/ventas/')

delta = spark.read.format('delta').load('s3://bucket/delta/ventas/')
delta.createOrReplaceTempView('ventas')
spark.sql("SELECT mes, SUM(total) FROM ventas GROUP BY mes").show()
'''
print(delta_demo.splitlines()[:15])

## 4. Buenas prácticas de Lakehouse

- Definir contratos de datos y versionado de esquemas.
- Gestionar tamaños de archivos y compaction (OPTIMIZE/VACUUM).
- Catalogación y políticas de acceso por dominio (Data Mesh).
- Observabilidad y linaje (OpenLineage/Marquez, DataHub).